Do the regression with another outcome variable: work effort, where we take workin overtime as the measure ("Do you work overtime?" -> Yes/No)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import math

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

In [2]:
# define path: insert the path where the SOEP data is stored on your computer here
from pathlib import Path
# Maxie: /Volumes/dohmen_soep/SOEP-CORE.v36eu_STATA/Stata/raw
# Max: C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/SOEP-Data/Stata/raw
data_folder = Path("/Volumes/dohmen_soep/SOEP-CORE.v36eu_STATA/Stata/raw")
# define relevant subsets of SOEP-data
file_names = ['vp', 'wp']

file_paths = [data_folder / f"{file_name}.dta" for file_name in file_names]
# some controls are in gen data
file_paths_2 = [data_folder / f"{file_name}gen.dta" for file_name in file_names]

In [4]:
data05 = pd.read_stata(file_paths[0], columns=["pid","hid", "syear","vp12602", "vp12603", "vp12605"]).set_index(['pid', 'hid'])
df_05 = data05.rename(columns={ 'vp12602': 'take_revenge', 'vp12603': 'similar_problems', 'vp12605': 'insult_back'})

In [5]:
data06 = pd.read_stata(file_paths[1], columns=["pid", "hid", "syear","wp12402", "wp12401", "wp55", "wp5902", 'wp43b01', "wp43b03", "wp43b05", "wp43b07"]).set_index(['pid', 'hid'])
df_06 = data06.rename(columns={"wp12401": "gender", "wp12402": "year_birth", 'wp43b01': "recog_sup", "wp43b03": "recog_effort", "wp43b05": "recog_personal", "wp43b07": "recog_pay", "wp5902": "wage_lastmonth", "wp55": "overtime"})
#'wp43b02': 'felt_recog_sup', 'wp43b04': 'felt_recog_effort', 'wp43b06': 'felt_recog_pay', "wp43b03": "recog_effort", "wp43b05": "recog_personal", "wp43b07": "recog_pay"
#"wp0102": "satisfaction_work"

In [6]:
hours06 = pd.read_stata(file_paths_2[1], columns=["pid","hid", "syear", "wvebzeit", "betr06", "wpsbil", "nace06", "werwzeit", "wbilzeit"]).set_index(['pid', 'hid'])
work06 = hours06.rename(columns={'wvebzeit': 'working_hours', "betr06": "firmsize", "wpsbil": "school_degree", "nace06": "sector", "werwzeit": "tenure" , "wbilzeit" : "years_educ"})

In [7]:
# mapping for reciprocity questions: same scale for all
reciprocity_questions_mapping = {
    '[1] Trifft ueberhaupt nicht zu': 1,
    '[2] Skala 1-7': 2,
    '[3] Skala 1-7': 3,
    '[4] Skala 1-7': 4,
    '[5] Skala 1-7': 5,
    '[6] Skala 1-7': 6,
    '[7] Trifft voll zu': 7,
    '[-1] keine Angabe': -1,
}

overtime_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Ja': 1,
    '[2] Nein': 2,
    '[3] T.n.z., Selbstaendiger': 3,
}
## mapping for recognition questions: binary -> unfair treatment: No -> later 1 fair treatment: Yes -> 0
recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Ja': 2,
    '[2] Nein': 1,
}
# felt recog mapping
felt_recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Gar nicht': 1,
    '[2] Maessig': 2,
    '[3] Stark': 3,
    '[4] Sehr stark': 4,
}
# mapping for firmsize -> we need to recode this in a sensible way: jumps are the same: first change: selbstständig to 0
firmsize_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Unter  5': 1,
    '[2] 5 bis 10': 2,
    '[3] 11 bis unter 20': 3,
    '[4] bis 90: unter 20': 4,
    '[5] 91-04: 5 bis unter 20': 5,
    '[6] 20 bis unter 100': 6,
    '[7] 100 bis unter 200': 7,
    '[8] bis 98: 20 bis unter 200': 8,
    '[9] 200 bis unter 2000': 9,
    '[10] 2000 und mehr': 10,
    '[11] Selbstaendig-ohne Mitarb.': 0,
}

# mapping for sectors: only to easier remove negatives
sector_map = {
    "[1] Landwirtschaft und  Jagd": 1,
    "[2] Forstwirtschaft": 2,
    "[5] Fischerei und Fischzucht": 5,
    "[10] Kohlenbergbau, Torfgewinnung": 10,
    "[11] Gewinnung von Erdöl und Erdgas, Erbringung damit verbundener Dienstleistungen": 11,
    "[12] Bergbau auf Uran- und Thoriumerze": 12,
    "[13] Erzbergbau": 13,
    "[14] Gewinnung von Steinen und Erden, sonstiger Bergbau": 14,
    "[15] Herstellung von Nahrungs- und Futtermitteln sowie Getränken": 15,
    "[16] Tabakverarbeitung": 16,
    "[17] Herstellung von Textilien": 17,
    "[18] Herstellung von Bekleidung": 18,
    "[19] Herstellung von Leder und Lederwaren": 19,
    "[20] Herstellung von Holz sowie Holz-, Kork- und Flechtwaren (ohne Herstellung von Möbeln)": 20,
    "[21] Herstellung von Papier, Pappe und Waren daraus": 21,
    '[22] Herstellung von Verlags- und Druckerzeugnissen,  Vervielfältigung von bespielten Ton-, Bild- und Datenträgern': 22,
    "[23] Kokerei, Mineralölverarbeitung, Herstellung und Verarbeitung von Spalt- und Brutstoffen": 23,
    "[24] Herstellung von chemischen Erzeugnissen": 24,
    "[25] Herstellung von Gummi- und Kunststoffwaren": 25,
    "[26] Herstellung von Glas und Glaswaren, Keramik, Verarbeitung von Steinen und Erden": 26,
    "[27] Metallerzeugung und -bearbeitung": 27,
    "[28] Herstellung von Metallerzeugnissen": 28,
    "[29] Maschinenbau": 29,
    "[31] Herstellung von Geräten der Elektrizitätserzeugung, -verteilung u. Ä.": 31,
    "[30] Herstellung von Büromaschinen, Datenverarbeitungsgeräten und -einrichtungen": 30,
    "[32] Rundfunk- und Nachrichtentechnik": 32,
    "[33] Medizin-, Mess-, Steuer- und Regelungstechnik, Optik, Herstellung von Uhren": 33,
    "[34] Herstellung von Kraftwagen und Kraftwagenteilen": 34,
    "[35] Sonstiger Fahrzeugbau": 35,
    "[36] Herstellung von Möbeln, Schmuck, Musikinstrumenten, Sportgeräten, Spielwaren und sonstigen Erzeugnissen": 36,
    "[37] Rückgewinnung": 37,
    "[40] Energieversorgung": 40,
    "[41] Wasserversorgung": 41,
    "[45] Bau": 45,
    "[50] Kraftfahrzeughandel; Instandhaltung und Reparatur von Kraftfahrzeugen; Tankstellen": 50,
    "[51] Handelsvermittlung und Großhandel (ohne Handel mit Kraftfahrzeugen)": 51,
    "[52] Einzelhandel (ohne Handel mit Kraftfahrzeugen und ohne Tankstellen); Reparatur von Gebrauchsgütern": 52,
    "[55] Beherbergungs- und Gaststätten": 55,
    "[60] Landverkehr; Transport in Rohrfernleitungen": 60,
    "[61] Schifffahrt": 61,
    "[62] Luftfahrt": 62,
    "[63] Hilfs- und Nebentätigkeiten für den Verkehr; Verkehrsvermittlung": 63,
    "[64] Nachrichtenübermittlung": 64,
    "[65] Kreditinstitute": 65,
    "[66] Versicherungen (ohne Sozialversicherung)": 66,
    "[67] Mit den Kreditinstituten und Versicherungen verbundene Tätigkeiten": 67,
    "[70] Grundstücks- und Wohnungswesen": 70,
    "[71] Vermietung beweglicher Sachen ohne Bedienungspersonal": 71,
    "[72] Datenverarbeitung und Datenbanken": 72,
    "[73] Forschung und Entwicklung": 73,
    "[74] Erbringung von unternehmensbezogenen Dienstleistungen": 74,
    "[75] Öffentliche Verwaltung, Verteidigung, Sozialversicherung": 75,
    "[80] Erziehung und Unterricht": 80,
    "[85] Gesundheits-, Veterinär- und Sozialwesen": 85,
    "[90] Abwasser- und Abfallbeseitigung und sonstige Entsorgung": 90,
    "[91] Interessenvertretungen sowie kirchliche und sonstige Vereinigungen (ohne Sozialwesen, Kultur und Sport)": 91,
    "[92] Kultur, Sport und Unterhaltung": 92,
    "[93] Erbringung von sonstigen Dienstleistungen": 93,
    "[95] Private Haushalte mit Hauspersonal": 95,					
    "[96] Industrie - ohne weitere Zuordnung": 96,					
    "[97] Handwerk - ohne weitere Zuordnung": 97,					
    "[98] Dienstleistungen ohne weitere Zuordnung": 98,					
    "[99] Exterritoriale Organisationen und Körperschaften": 99,				
    "[100] Produzierendes Gewerbe ohne w.Zuordnung": 100,
    "[-1] keine Angabe": -1,
    '[-2] trifft nicht zu': -2, 
    "[-3] unplausibler Wert": -3,
    "[-4] unzulaessige Mehrfachantwort": -4, 
    "[-5] in Fragebogenversion nicht enthalten": -5,
    "[-6] Fragebogenversion mit geaenderter Filterfuehrung": -6, 
    "[-7] nur in weniger eingeschraenkter Edition verfuegbar": -7,
    "[-8] Frage in diesem Jahr nicht Teil des Frageprogramms": -8,
}
# mapping for school degree: to easier remove negatives
school_degree_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[1] Hauptschulabschluss': 1,
    '[2] Realschulabschluss': 2,
    '[3] Fachhochschulreife': 3,
    '[4] Abitur': 4,
    '[5] Anderer Abschluss': 5,
    '[6] Ohne Abschluss verlassen': 6,
    '[7] Noch kein Abschluss': 7,
    '[8] Keine Schule besucht': 8,
}
# reversed mapping to redo changes

reversed_mapping_sector = {v: k for k, v in sector_map.items()}
reversed_mapping_schoold = {v: k for k, v in school_degree_mapping.items()}

In [8]:

def recode_categoricals(inputdf):

    merged = inputdf  
    
    # recode Gender variable
    merged['gender'].replace('[2] Weiblich', 2,inplace=True)
    merged['gender'].replace('[1] Maennlich', 1,inplace=True)
    # recode reciprocity variables
    merged[["similar_problems","take_revenge","insult_back"]] = merged[["similar_problems","take_revenge","insult_back"]].apply(lambda x: x.map(reciprocity_questions_mapping))
    # recode recognition variables
    merged[["recog_sup","recog_effort","recog_personal","recog_pay"]] = merged[["recog_sup","recog_effort","recog_personal","recog_pay"]].apply(lambda x: x.map(recog_mapping))
    # recode felt recognition variables
    #merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]] = merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]].apply(lambda x: x.map(felt_recog_mapping))
    # recode firm size
    merged['firmsize'] = merged['firmsize'].map(firmsize_mapping)
    # recode new job reason variable
   # merged['reason_new_job'] = merged['reason_new_job'].map(reason_new_job_mapping)
    # recode job change variable
   # merged['new_job']= merged['new_job'].map(new_job_mapping)
    # recode turnover intention variable
    #merged['turnover_intention'] = merged['turnover_intention'].map(turnover_mapping)
    # recode sector
    merged['sector'] = merged['sector'].map(sector_map)
    # recode school degree
    merged['school_degree'] = merged['school_degree'].map(school_degree_mapping)
    merged['overtime'] = merged['overtime'].map(overtime_mapping)
    output = merged
    return output

In [9]:
# Merge dataframes: a bit tough to read as its nested, merges 3 dataframes: 2005,2006,and 2006gen
allmerged_df = pd.merge(pd.merge(df_05, df_06, on=['pid', 'hid']), work06, on=['pid', 'hid'])
recoded = recode_categoricals(allmerged_df).astype('int')

# construct indsutry-relative wage
sector_wage_averages = recoded.groupby('sector')['wage_lastmonth'].mean()
recoded["sector_avg_wage"] = recoded["sector"].map(sector_wage_averages)
recoded["relative_wage"] = recoded["wage_lastmonth"] / recoded["sector_avg_wage"]
# replaces negative values with n.a.n 
recoded = recoded.mask(recoded < 0, np.nan) 
# construct avg reciprocity measure
recoded['avg_rec'] = recoded[['take_revenge', 'similar_problems', 'insult_back']].mean(axis=1)
# construct age, potential experience and age^2
recoded['age'] = 2006 - recoded['year_birth']
recoded["potential_experience"] = pow((recoded["age"] - 18), 2)
recoded["age_squared"] = (recoded["age"] ** 2) / 100
recoded["tenure_squared"] = (recoded["tenure"] ** 2) / 100
# recode categoricals back to make it better readable
#recoded["reason_new_job"] = recoded["reason_new_job"].map(reversed_mapping_reason)
recoded["sector"]=recoded["sector"].map(reversed_mapping_sector)
recoded["school_degree"] = recoded["school_degree"].map(reversed_mapping_schoold)

# transform binary variables with 1 and 2 into 1 and 0
columns_to_transform = ["recog_sup","recog_effort", "recog_pay", "recog_personal" ,"gender", "overtime"]

# Iterate over the columns and replace the values 2 with 0
for col in columns_to_transform:
    recoded[col] = recoded[col].replace({2: 0})

# save df somewhere so its not muted when repeatedly executing this cell: Can later transform that into functions
dfnan = recoded
# remove rows with '3' in overtime
dfnan.drop(dfnan[dfnan['overtime'] == 3].index, inplace=True)

dfnan.head(3)

,,syear_x,take_revenge,similar_problems,insult_back,syear_y,year_birth,gender,overtime,wage_lastmonth,recog_sup,...,sector,tenure,years_educ,sector_avg_wage,relative_wage,avg_rec,age,potential_experience,age_squared,tenure_squared
pid,hid,,,,,,,,,,,,,,,,,,,,,
201,27,2005,1.0,5.0,1.0,2006,1926,0,NaN,NaN,NaN,...,NaN,NaN,10.0,94.115693,NaN,2.333333,80,3844,64.00,NaN
901,94,2005,5.0,3.0,3.0,2006,1951,0,1.0,1252.0,1.0,...,[52] Einzelhandel (ohne Handel mit Kraftfahrze...,5.0,10.0,943.961012,1.326326,3.666667,55,1369,30.25,0.25
1202,124,2005,1.0,1.0,1.0,2006,1913,0,NaN,NaN,NaN,...,NaN,NaN,10.0,94.115693,NaN,1.000000,93,5625,86.49,NaN


### Mincer Wage Regression

In [10]:
df_mincer = dfnan.drop(columns=['syear_x', 'similar_problems', 'take_revenge', 'insult_back','syear_y',
       'recog_personal', 'recog_pay', 'syear_y', 'year_birth','sector_avg_wage', 'school_degree','overtime', 'recog_sup', 'age', 'relative_wage', 'recog_effort', 'working_hours', 'syear', 'tenure_squared'])

In [11]:
# Convert 'gender' and 'sector' columns to categorical data type
for col in ['gender', 'sector']:
    df_mincer[col] = df_mincer[col].astype('category')

In [12]:
df_mincer = df_mincer.dropna()

In [13]:
# Define the dependent variable
y = df_mincer['wage_lastmonth']

# Define the independent variables
X = df_mincer[['gender', 'firmsize', 'tenure', 'years_educ', 'potential_experience', 'age_squared']]

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Fit the Mincer wage regression model
mincer_model = sm.OLS(y, X).fit()

# Print the summary statistics of the model
#mincer_model.summary()

# Get the residuals of the model
residuals_mincer = mincer_model.resid

In [14]:
residuals_mincer

pid      hid   
901      94        533.727630
1701     175      -426.096690
1704     175       815.126244
2301     230       434.304466
2302     65889     676.697090
                      ...    
8250402  825042    501.363151
8250702  825077   -191.377937
8250802  825085   -992.816723
8253302  825336   -221.751951
8256802  825689    444.629804
Length: 7718, dtype: float64

In [15]:
df_regression = dfnan.drop(columns=['syear_x', 'similar_problems', 'take_revenge', 'insult_back','syear_y', 'recog_sup',
       'recog_personal', 'recog_pay', 'syear_y', 'year_birth', 'sector_avg_wage', 'sector', 'wage_lastmonth', 'school_degree', "syear"])

In [16]:
# Create a new column in the dataframe with the same name as the residuals array
df_regression['relative_wage'] = None

# Match the rows of the dataframe with the values in the residuals array using the index
df_regression.loc[df_regression.index, 'relative_wage'] = residuals_mincer

# Rename the column "relative_wage" to "mincer_residuals"
df_regression = df_regression.rename(columns={'relative_wage': 'mincer_residuals'})



/var/folders/7l/hw861dtx52z8nxgtzjvpp8_h0000gn/T/ipykernel_928/2021374678.py:5: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_regression.loc[df_regression.index, 'relative_wage'] = residuals_mincer


In [17]:
df_regression["interaction"] = df_regression["recog_effort"] * df_regression["avg_rec"]

In [18]:
df_regression = df_regression.dropna()

In [19]:
df_regression.columns

Index(['gender', 'overtime', 'recog_effort', 'working_hours', 'firmsize',
       'tenure', 'years_educ', 'mincer_residuals', 'avg_rec', 'age',
       'potential_experience', 'age_squared', 'tenure_squared', 'interaction'],
      dtype='object')

In [22]:
from statsmodels.discrete.discrete_model import Probit

Y = df_regression["overtime"]
X = df_regression.drop(columns=["overtime"])
X = sm.add_constant(X)
model = Probit(Y, X.astype(float))
probit_model = model.fit(cov_type= "HC3")

print(probit_model.summary())
print(probit_model.summary().as_latex())

         Current function value: 0.488249
         Iterations: 35
                          Probit Regression Results                           
Dep. Variable:               overtime   No. Observations:                 6882
Model:                         Probit   Df Residuals:                     6868
Method:                           MLE   Df Model:                           13
Date:                Mon, 09 Jan 2023   Pseudo R-squ.:                  0.1016
Time:                        18:51:01   Log-Likelihood:                -3360.1
converged:                      False   LL-Null:                       -3740.2
Covariance Type:                  HC3   LLR p-value:                4.582e-154
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.2627    1.5e+05  -1.75e-06      1.000   -2.95e+05    2.95e+05
gender                   0.1402    

/Users/maxieschulze/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Now, instead of using a probit model, we use OLS.

In [25]:
from statsmodels.regression.linear_model import OLS

Y = df_regression["overtime"]
X = df_regression.drop(columns=["overtime"])
X = sm.add_constant(X)
model = OLS(Y, X.astype(float))
lpm_model = model.fit(cov_type= "HC3")

lpm_model.summary()
#print(lpm_model.summary().as_latex())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               overtime   R-squared:                       0.105
Model:                            OLS   Adj. R-squared:                  0.104
Method:                 Least Squares   F-statistic:                     2633.
Date:                Mon, 09 Jan 2023   Prob (F-statistic):               0.00
Time:                        19:04:27   Log-Likelihood:                -3462.3
No. Observations:                6882   AIC:                             6951.
Df Residuals:                    6869   BIC:                             7040.
Df Model:                          12                                         
Covariance Type:                  HC3                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.0239      0.006     -3.802      0.000      -0.036      -0.012
gender                   0.0228      0.011      2.072      0.038       0.001       0.044
recog_effort             0.1162      0.024      4.815      0.000       0.069       0.164
working_hours            0.0078      0.001     10.767      0.000       0.006       0.009
firmsize                 0.0062      0.002      3.064      0.002       0.002       0.010
tenure                  -0.0001      0.002     -0.074      0.941      -0.003       0.003
years_educ               0.0244      0.002     14.070      0.000       0.021       0.028
mincer_residuals      5.171e-05   6.42e-06      8.059      0.000    3.91e-05    6.43e-05
avg_rec                  0.0027      0.005      0.556      0.578      -0.007       0.012
age                     -0.0123      0.005     -2.421      0.015      -0.022      -0.002
potential_experience    -0.0010      0.000     -4.758      0.000      -0.001      -0.001
age_squared              0.0731      0.019      3.930      0.000       0.037       0.110
tenure_squared          -0.0016      0.005     -0.337      0.736      -0.011       0.008
interaction             -0.0119      0.007     -1.668      0.095      -0.026       0.002
==============================================================================
Omnibus:                      954.941   Durbin-Watson:                   1.883
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1380.357
Skew:                          -1.088   Prob(JB):                    1.82e-300
Kurtosis:                       2.722   Cond. No.                     6.58e+16
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The smallest eigenvalue is 1.27e-24. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""